In [2]:
import torch
from transformers import AutoModel, AutoTokenizer
import re
from vncorenlp import VnCoreNLP
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
phobert = AutoModel.from_pretrained("vinai/phobert-base", output_hidden_states = True)


Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [7]:
from test import translate_doc

In [3]:
annotator = VnCoreNLP("C:\\Users\\Dell 7577\\Downloads\\VnCoreNLP-master\\VnCoreNLP-1.1.1.jar", annotators="wseg, pos, ner", max_heap_size='-Xmx2g')


In [4]:

# tokenize sentences
def tokenize_sentences(doc):
    encoded_sentences = tokenizer(doc, padding=True, truncation=True, return_tensors='pt')
    # encoded_sentences = []
    # for sen in doc: 
    #     encoded_sentences.append(tokenizer(sen, return_tensors='pt')['input_ids'])
    
    return encoded_sentences['input_ids'], encoded_sentences['attention_mask']
    #return encoded_sentences
def read_file(file_path):
    res = []
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        res = [re.sub('\n', '', line).strip() for line in lines]
        punctuation = '[!"#$%&\'()*+,./;<=>?@[\\]^_`{|}~]'
        res = [re.sub(punctuation, '', line) for line in res]
        
    return res
        



In [8]:
src_file = './khmer_1000_sens.txt'
trans_file = './data_vi_samples_trans.txt'

In [10]:
translate_doc(src_file, trans_file)

  0%|          | 0/28 [00:00<?, ?it/s]finding source element...
finding trans element...
  4%|▎         | 1/28 [00:41<18:28, 41.04s/it]finding source element...
finding trans element...
  7%|▋         | 2/28 [01:10<14:54, 34.40s/it]finding source element...
finding trans element...
 11%|█         | 3/28 [01:45<14:28, 34.73s/it]finding source element...
finding trans element...
 14%|█▍        | 4/28 [02:13<12:44, 31.85s/it]finding source element...
finding trans element...
 18%|█▊        | 5/28 [02:42<11:53, 31.02s/it]finding source element...
finding trans element...
 21%|██▏       | 6/28 [03:14<11:24, 31.10s/it]finding source element...
finding trans element...
 25%|██▌       | 7/28 [03:52<11:41, 33.42s/it]finding source element...
finding trans element...
 29%|██▊       | 8/28 [04:28<11:23, 34.18s/it]finding source element...
finding trans element...
 32%|███▏      | 9/28 [04:55<10:09, 32.08s/it]finding source element...
finding trans element...
 36%|███▌      | 10/28 [05:35<10:23, 3

In [5]:
trans_doc = read_file('./data_vi_samples_trans.txt')
vi_doc = read_file('./vie_doc.txt')
km_doc = read_file('./data_khm_samples.txt')


In [5]:
vi_doc = vi_doc[:len(trans_doc)]

In [5]:
len(trans_doc)

1878

In [6]:
from numpy import random
def generate_labels(trans_doc, vi_doc):
    positive_set = []
    negative_set = set()
    for i in range(len(trans_doc)):
        positive_set.append((i, i))
    cnt = 0
    while cnt != len(positive_set):
        i = random.randint(len(trans_doc))
        j = random.randint(len(vi_doc))
        if i == j: 
            continue
        cnt += 1
        negative_set.add((i, j))
    positive_labels = [1 for pair in positive_set]
    negative_labels = [0 for pair in negative_set]
    return positive_set, positive_labels, list(negative_set), negative_labels
                

In [7]:
positive_examples, positive_labels, negative_examples, negative_labels = generate_labels(trans_doc, vi_doc)

In [16]:
len(negative_examples)

1878

In [8]:
def segment_sentence(sen):
    word_segmented_sen = annotator.tokenize(sen)
    # word_segmented_sen = annotator.annotate(sen)['sentences'][0]
    res = ''
    # for word in word_segmented_sen:
    #     if word['nerLabel'] not in ['B-PER', 'B-LOC', 'B-ORG', 'B-MISC']:
    #         res += word['form'] + ' '
    for sen in word_segmented_sen:
        for word in sen:
            res += word + ' '
    return res.strip().lower() 
def concatenate_hidden_states(embed):
    return torch.cat(embed[2][-4:], axis = 1)
def cosine_similarity(embed1, embed2):
    return torch.dot(embed1.reshape(768), embed2.reshape(768))/(torch.norm(embed1) * torch.norm(embed2))

def compare_similarity(doc1, doc2):
    match_table = []
    for i, sen1 in enumerate(doc1):
        match_table.append([])
        sen1_embed = phobert()
        for j, sen2 in enumerate(doc2):
            # 4 last layers
            similarity = cosine_similarity(torch.mean(concatenate_hidden_states(sen1), dim = 1), torch.mean(concatenate_hidden_states(sen2), dim = 1))
            # last layer
            # similarity = cosine_similarity(torch.mean(sen1[0], dim = 1), torch.mean(sen2[0], dim = 1))
            # output of 
            #similarity = cosine_similarity(sen1[1], sen2[1])
            match_table[i].append(similarity)
    return match_table


In [9]:
vi_doc_segmented = []
trans_doc_segmented = []
for sen1, sen2 in zip(vi_doc, trans_doc): 
    vi_doc_segmented.append(segment_sentence(sen1))
    trans_doc_segmented.append(segment_sentence(sen2))

In [10]:
for sen1, sen2 in zip(trans_doc_segmented, vi_doc_segmented):
    print('Original: ' + sen2)
    print('Translation: ' + sen1)
    print('-'*50)

Original: ý đã đánh_bại bồ_đào_nha với tỉ_số 31-5 ở bảng c giải vô_địch rugby thế_giới 2007 tại parc des princes_pari_pháp
Translation: ý đánh_bại bồ_đào_nha với tỷ_số 31-5 ở bảng c của giải bóng_bầu_dục thế_giới 2007 tại paris_pháp
--------------------------------------------------
Original: andrea_maisi đã mở tỉ_số cho ý ở phút thứ tư với một quả try
Translation: entramas ghi_bàn ở phút thứ tư của hiệp một cho ý
--------------------------------------------------
Original: chiếm thế áp_đảo trong hầu_hết hiệp đầu nhưng ý đã không_thể ghi thêm try nào trước khi nghỉ giữa giờ tuy_nhiên david_bortolussi đã sút ba quả phạt_đền kéo_dài thế dẫn_đầu của họ
Translation: dù lấn_lướt trong nửa đầu hiệp một italia không ghi thêm được điểm nào trước giờ nghỉ nhưng david_botulusi đã ghi liền ba bàn từ chấm_phạt_đền mở_rộng cách_biệt
--------------------------------------------------
Original: bồ_đào_nha chưa bao_giờ từ_bỏ và david_penalva đã ghi một try ở phút thứ 33 điểm duy_nhất của họ trong trận

In [10]:
trans_doc_ids, trans_doc_mask = tokenize_sentences(trans_doc_segmented)
vi_doc_ids, vi_doc_mask = tokenize_sentences(vi_doc_segmented)
# trans_doc_ids = tokenize_sentences(trans_doc_segmented)
# vi_doc_ids = tokenize_sentences(vi_doc_segmented)
print(trans_doc_ids.shape)
print(vi_doc_ids.shape)


torch.Size([1878, 98])
torch.Size([1878, 133])


In [11]:
from tqdm import tqdm

In [18]:
def get_doc_embedding(doc_ids, doc_mask):
    print(phobert(doc_ids, attention_mask=doc_mask)[0].shape)


In [19]:
get_doc_embedding(trans_doc_ids, trans_doc_mask)

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 565383168 bytes. Buy new RAM!


In [ ]:
def get_score(vi_doc_ids, vi_doc_mask, trans_doc_ids, trans_doc_mask):
    vi_doc_embed = phobert(vi_doc_ids, attention_mask = vi_doc_mask)
    print(vi_doc_embed)
    trans_doc_embed = phobert(trans_doc_ids, attention_mask=trans_doc_mask)



In [21]:

from tqdm import tqdm
def get_similarity_score(data):
    # cnt = 0
    res = []
    for pair in tqdm(data):
        # cnt += 1
        # print(cnt)
        trans_index = pair[0]
        vi_index = pair[1]
        trans_id = trans_doc_ids[trans_index]
        trans_mask = trans_doc_mask[trans_index]
        vi_id = vi_doc_ids[vi_index]
        vi_mask = vi_doc_mask[vi_index]
        # trans_embed = phobert(trans_id.reshape(1, -1))
        # vi_embed = phobert(vi_id.reshape(1, -1))

        trans_embed = phobert(trans_id.reshape(1, -1), attention_mask=trans_mask.reshape(1, -1))
        vi_embed = phobert(vi_id.reshape(1, -1), attention_mask=vi_mask.reshape(1, -1))
        res.append(float(cosine_similarity(torch.mean(trans_embed[0], dim = 1), torch.mean(vi_embed[0], dim = 1))))
    return res


In [22]:
positive_score = get_similarity_score(positive_examples)
negative_score = get_similarity_score(negative_examples)

100%|██████████| 1878/1878 [13:40<00:00,  2.29it/s]


In [23]:
cnt = 0
for i, score in enumerate(negative_score):
    if score > 0.9:
        cnt += 1
        print('Original ' + str(negative_examples[i][1]) + ': ' + vi_doc_segmented[negative_examples[i][1]])
        print('Translated: ' + str(negative_examples[i][0]) + ': ' + trans_doc_segmented[negative_examples[i][0]])
        print(score)
        print('-'*100)

Original 926: bà tuyên_bố đã bị lừa vào ở trên bất_chấp những nghi_ngờ của mình về chiến_tranh và nói với uỷ_ban rằng nội_các trong đó bà là một thành_viên không phải là một tổ_chức ra quyết_định và rằng quốc_hội chỉ đơn_giản là một bù_nhìn
Translated: 807: cô gái cho rằng lengham đã lấy đi trinh_tiết của cô năm 14 tuổi nhưng thẩm_phán statman nói : crown đã thừa_nhận rằng không có bằng_chứng nào hỗ_trợ cho những tội_ác đặc_biệt này
0.9134535193443298
----------------------------------------------------------------------------------------------------
Original 1623: thủ_tướng pháp francois_fillon đã nói rằng lệnh cấm là phù_hợp với các giá_trị quốc_gia của pháp
Translated: 85: bộ_trưởng nội_vụ renman_malik đã xác_nhận rằng hoạt_động khủng_bố như_vậy đã khiến chính_phủ mở chiến_dịch chống lại taliban hoạt_động ở nam_waziristan
0.9052777886390686
----------------------------------------------------------------------------------------------------


In [25]:
cnt = 0
for i, score in enumerate(positive_score):
    if score < 0.7:
        cnt += 1
        print('Original ' + str(positive_examples[i][1]) + ': ' + vi_doc_segmented[positive_examples[i][1]])
        print('Translated: ' + str(positive_examples[i][0]) + ': ' + trans_doc_segmented[positive_examples[i][0]])
        print(score)
        print('-'*100)

Original 2: chiếm thế áp_đảo trong hầu_hết hiệp đầu nhưng ý đã không_thể ghi thêm try nào trước khi nghỉ giữa giờ tuy_nhiên david_bortolussi đã sút ba quả phạt_đền kéo_dài thế dẫn_đầu của họ
Translated: 2: dù lấn_lướt trong nửa đầu hiệp một italia không ghi thêm được điểm nào trước giờ nghỉ nhưng david_botulusi đã ghi liền ba bàn từ chấm_phạt_đền mở_rộng cách_biệt
0.44781845808029175
----------------------------------------------------------------------------------------------------
Original 5: tuy_nhiên bortolussi đã ghi cú phạt_đền lần thứ tư của trận đấu tiếp đó là những quả try của mauro_bergammasco và quả try thứ hai của andrea_masi chốt lại chiến_thắng cho ý
Translated: 5: tuy_nhiên botulusi đã ghi_bàn thắng thứ tư của trận đấu từ chấm_phạt_đền sau đó morobi gama-masco và entramas ghi_bàn mang về chiến_thắng cho ý
0.5185046792030334
----------------------------------------------------------------------------------------------------
Original 14: bà nhấn_mạnh rằng các biện_pháp an_

In [26]:
cnt

308

In [64]:
def get_word_similarity(word1, word2):
    segmented_word1 = segment_sentence(word1)
    segmented_word2 = segment_sentence(word2)
    print(segmented_word1)
    print(segmented_word2)
    segmented_word1_id = tokenizer(segmented_word1, return_tensors='pt')['input_ids']
    segmented_word2_id = tokenizer(segmented_word2, return_tensors='pt')['input_ids']
    embed1 = phobert(segmented_word1_id)
    embed2 = phobert(segmented_word2_id)
    # print(torch.mean(embed1[0], dim = 1).shape)
    # print(torch.mean(embed2[0], dim = 1).shape)
    return float(cosine_similarity(torch.mean(embed1[0], dim = 1), torch.mean(embed2[0], dim = 1)))

In [21]:
get_word_similarity('tiến cử', 'đề cử')

tiến_cử
đề_cử


0.49769875407218933

In [27]:
from nltk.translate.bleu_score import sentence_bleu, modified_precision, brevity_penalty
from nltk.translate.nist_score import sentence_nist
import pyter 


In [28]:

def get_bleu_score(candidate, reference, ngrams):
    candidate = candidate.split(' ')
    reference = [reference.split(' ')]
    return float(modified_precision(reference, candidate, ngrams))
    #return sentence_bleu(reference, candidate, smoothing_function=chencherry.method1)
def get_nist_score(candidate, reference, ngrams):
    candidate = candidate.split(' ')
    reference = [reference.split(' ')]
    return float(sentence_nist(reference, candidate, ngrams))
def get_ter_score(candidate, reference):
    return pyter.ter(candidate.split(), reference.split())


In [40]:
positive_bleu_scores = {'1-gram': [], '2-gram': [], '3-gram': [], '4-gram': []}
positive_nist_scores = {'1-gram': [], '2-gram': [], '3-gram': [], '4-gram': [], '5-gram': []}
positive_ter_scores = []
for pair in positive_examples:
    trans_index = pair[0]
    vi_index = pair[1]
    trans_sen = trans_doc_segmented[trans_index]
    vi_sen = vi_doc_segmented[vi_index]
    positive_bleu_scores['1-gram'].append(get_bleu_score(trans_sen, vi_sen, 1))
    positive_bleu_scores['2-gram'].append(get_bleu_score(trans_sen, vi_sen, 2))
    positive_bleu_scores['3-gram'].append(get_bleu_score(trans_sen, vi_sen, 3))
    positive_bleu_scores['4-gram'].append(get_bleu_score(trans_sen, vi_sen, 4))
    positive_nist_scores['1-gram'].append(get_nist_score(trans_sen, vi_sen, 1))
    positive_nist_scores['2-gram'].append(get_nist_score(trans_sen, vi_sen, 2))
    positive_nist_scores['3-gram'].append(get_nist_score(trans_sen, vi_sen, 3))
    positive_nist_scores['4-gram'].append(get_nist_score(trans_sen, vi_sen, 4))
    positive_nist_scores['5-gram'].append(get_nist_score(trans_sen, vi_sen, 5))
    positive_ter_scores.append(get_ter_score(trans_sen, vi_sen))

In [41]:
negative_bleu_scores = {'1-gram': [], '2-gram': [], '3-gram': [], '4-gram': []}
negative_nist_scores = {'1-gram': [], '2-gram': [], '3-gram': [], '4-gram': [], '5-gram': []}
negative_ter_scores = []
for pair in negative_examples:
    trans_index = pair[0]
    vi_index = pair[1]
    trans_sen = trans_doc_segmented[trans_index]
    vi_sen = vi_doc_segmented[vi_index]
    negative_bleu_scores['1-gram'].append(get_bleu_score(trans_sen, vi_sen, 1))
    negative_bleu_scores['2-gram'].append(get_bleu_score(trans_sen, vi_sen, 2))
    negative_bleu_scores['3-gram'].append(get_bleu_score(trans_sen, vi_sen, 3))
    negative_bleu_scores['4-gram'].append(get_bleu_score(trans_sen, vi_sen, 4))
    negative_nist_scores['1-gram'].append(get_nist_score(trans_sen, vi_sen, 1))
    negative_nist_scores['2-gram'].append(get_nist_score(trans_sen, vi_sen, 2))
    negative_nist_scores['3-gram'].append(get_nist_score(trans_sen, vi_sen, 3))
    negative_nist_scores['4-gram'].append(get_nist_score(trans_sen, vi_sen, 4))
    negative_nist_scores['5-gram'].append(get_nist_score(trans_sen, vi_sen, 5))
    negative_ter_scores.append(get_ter_score(trans_sen, vi_sen))

In [46]:
positive_score_merged = list(zip(positive_score, positive_bleu_scores['1-gram'], positive_bleu_scores['2-gram'], positive_bleu_scores['3-gram']
, positive_bleu_scores['4-gram'], positive_nist_scores['1-gram'], positive_nist_scores['2-gram'], positive_nist_scores['3-gram'], positive_nist_scores['4-gram'], positive_nist_scores['5-gram'], positive_ter_scores))


In [47]:
negative_score_merged = list(zip(negative_score, negative_bleu_scores['1-gram'], negative_bleu_scores['2-gram'], negative_bleu_scores['3-gram']
, negative_bleu_scores['4-gram'], negative_nist_scores['1-gram'], negative_nist_scores['2-gram'], negative_nist_scores['3-gram'], negative_nist_scores['4-gram'], negative_nist_scores['5-gram'], negative_ter_scores))

In [48]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import numpy as np 

In [49]:
score_merged = np.array(positive_score + negative_score).reshape(-1, 1)
labels_merged= np.array(positive_labels + negative_labels)

In [53]:
score_merged.shape

(3756, 11)

In [51]:
score_merged = np.array(positive_score_merged + negative_score_merged)
labels_merged= np.array(positive_labels + negative_labels)

In [52]:
labels_merged.shape

(3756,)

In [54]:
X_train, X_test, y_train, y_test = train_test_split(score_merged, labels_merged, test_size=0.3, random_state = 28)

In [55]:
logistic_clf = LogisticRegression()
logistic_clf.fit(X_train, y_train)

LogisticRegression()

In [56]:
logistic_clf.score(X_test, y_test)

0.8864241348713399

In [57]:
y_pred = logistic_clf.predict(X_test)


In [58]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.98      0.90       563
           1       0.97      0.80      0.88       564

    accuracy                           0.89      1127
   macro avg       0.90      0.89      0.89      1127
weighted avg       0.90      0.89      0.89      1127



In [59]:
import pickle  

In [60]:
filename = 'logistic_model_more_data.sav'
pickle.dump(logistic_clf, open(filename, 'wb'))

In [3]:
import flask

In [4]:
flask

<module 'flask' from 'D:\\anaconda\\lib\\site-packages\\flask\\__init__.py'>